In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import csv
from dragon import dragonV
import os

# data_parsing

In [2]:
root_path = '/Users/ivory/Documents/github/Philadelphia/real_final_data/'
subject_list = ['pp01', 'pp02', 'pp009', 'pp085', 'pp086', 'pp087', 'pp088', 'pp089'] 
#front, rear 2가지 존재. 파일명 format : {gait_catergory}_{front || rear}.xlsx
gait_category_list = ['gait1', 'gait2', 'fast', 'preferred', 'reaction', 'slow', 'stroop', 'turn']
seleted_openpose_joint_idx_list = [8, 9, 10, 11, 12, 13, 14, 19, 20, 21, 22, 23, 24]

## xlsx -> nomalize -> lower joint -> frame & gt pair

In [3]:
all_subject_data = []
for each_subject in subject_list:
    #root/ppxx/
    each_subject_path = root_path + each_subject + '/'
    files = os.listdir(each_subject_path)

    ###xlsx_list###
    excel_name_list = [file for file in files if file.endswith('.xlsx')]
    
    ###gt_list####
    tmp = os.listdir(each_subject_path + 'gt/')
    gt_name_list = [file for file in tmp if file.endswith('.npy')]

    #ppxx/{each_excel_name}
    #하나의 엑셀 파일에 대한 반복문
    for each_excel in excel_name_list:
        ##xlsx -> list##
        frame_data_list = dragonV.xlsx2data(each_subject_path + each_excel)
        #print(f'{each_excel} : {len(frame_data_list[0])}')
        ##list -> normalize##
        norm_frame_data_list = dragonV.nomalize_data(frame_data_list)
        #print(f'{each_subject}->{each_excel} : {len(norm_frame_data_list[0])}')

        ##noramlize -> select lower joint pos##
        selected_norm_frame_data_list = dragonV.get_selected_joint_pos_frame_list(norm_frame_data_list, seleted_openpose_joint_idx_list)
        #print(f'{each_subject}->{each_excel} : {len(selected_norm_frame_data_list[0])}')
        
        ## 여기까진 왔음...
        ##frame data : gt pair##
        for each_gt in gt_name_list:
            #확장자명 제거
            each_gt_name = each_gt[:-4]
            if each_gt_name in each_excel:
                #print(f'{each_subject}:{each_excel} -> {each_gt_name}')
                right_gt_np = np.load(each_subject_path + '/gt/' + each_gt)
                right_gt_list = right_gt_np.tolist()

                data_gt_pair_list = dragonV.make_dataAndGtPair(selected_norm_frame_data_list, right_gt_list)
                all_subject_data.append(data_gt_pair_list)
            else:
                continue


In [4]:
d1 = len(all_subject_data)
d2 = len(all_subject_data[0])
d3 = len(all_subject_data[0][0])
d4_0 = len(all_subject_data[0][0][0])
d4_1 = len(all_subject_data[0][0][1])


In [5]:
print('video_num :',d1)
print('frame_num :',d2)
print('features || label :',d3)
print('d4_0 :',d4_0)
print('d4_1 :',d4_1)

video_num : 73
frame_num : 1063
features || label : 2
d4_0 : 26
d4_1 : 4


## list -> torch

In [6]:
video_num = 0
frame_num = 0
features_labels = 0
np.array(all_subject_data[video_num][frame_num][features_labels])

array([0.98548271, 0.92060491, 0.98759102, 0.96146204, 0.9881226 ,
       0.9740866 , 0.99484523, 0.98557527, 0.81086829, 0.91704873,
       0.70520791, 0.97594994, 0.71989963, 0.9997064 , 0.62189694,
       0.89097924, 0.9188043 , 0.9825343 , 0.73717281, 0.98529396,
       0.99225403, 0.97184447, 0.98237504, 0.9717276 , 0.99244709,
       0.97882186])

[
    [
        [features],
        [features],
        ......... ,
        [features]
    ],
    [
        [features],
        [features],
        ......... ,
        [features]
    ],
    [
        [features],
        [features],
        ......... ,
        [features]
    ],    
]

In [118]:
[
    [labels],
    [labels],
    [labels],
    [labels],
    ........,
    [labels]
]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1705757532.py, line 6)

In [184]:
t = all_subject_data[0]
t_len = len(t)
t_frame_list = []
t_label_list = []
for i in range(0, t_len):
    t_frame_list.append(t[i][0])
    t_label_list.append(t[i][1])

In [188]:
print(f'frame_len : {len(t_frame_list)}')
print(f'label_len : {len(t_label_list)}')

frame_len : 1063
label_len : 1063


In [196]:
np.array(t_frame_list[:9])

array([[0.98548271, 0.92060491, 0.98759102, 0.96146204, 0.9881226 ,
        0.9740866 , 0.99484523, 0.98557527, 0.81086829, 0.91704873,
        0.70520791, 0.97594994, 0.71989963, 0.9997064 , 0.62189694,
        0.89097924, 0.9188043 , 0.9825343 , 0.73717281, 0.98529396,
        0.99225403, 0.97184447, 0.98237504, 0.9717276 , 0.99244709,
        0.97882186],
       [0.98563035, 0.95936408, 0.98771679, 0.96165813, 0.98816563,
        0.97402027, 0.99489721, 0.9855716 , 0.81091135, 0.91741665,
        0.70516431, 0.95371315, 0.71994492, 0.99957975, 0.63497453,
        0.90563174, 0.92120914, 0.98594924, 0.73730697, 0.98518937,
        0.98968955, 0.97188273, 0.97990497, 0.97173477, 0.99492728,
        0.97883029],
       [0.98572877, 0.92044172, 0.98767487, 0.9613836 , 0.9881226 ,
        0.97385442, 0.99487122, 0.98558631, 0.82275091, 0.91665252,
        0.70520791, 0.95289556, 0.71999022, 0.99959702, 0.63502042,
        0.90579095, 0.92121776, 0.9859887 , 0.73735169, 0.98525213,
      

In [193]:
print(len(t_frame_list[0]))
print(len(t_label_list[0]))

26
4


In [189]:
import copy

In [60]:
print(len(all_subject_data))
print(len(all_subject_data[0]))
print(len(all_subject_data[0][0][0]))

73
1063
26


In [83]:
# timesteps이 9라서 앞, 뒤로 레이블 데이터 4개씩 버려야 함.
'''
each_video : frame_num x (features || label) x (26 || 4)
'''
trimmed_data = []
trimmed_label = []

for each_video in all_subject_data:
    #각 비디오의 전체 프레임 길이
    video_len = len(each_video)
    #frame_number x each_element(26)
    total_frame_features_list = []
    #frame_number x each_label(4)
    total_frame_label_list = []
    
    #각각의 비디오에서 frame_data와 frame별 label 값 추출
    for i in range(0, video_len):
        total_frame_features_list.append(each_video[i][0])
        total_frame_label_list.append(each_video[i][1])

    #timesteps 수만큼 frame data 묶기.
    for idx in range(0, len(total_frame_features_list) - 8):
        trimmed_data.append(total_frame_features_list[idx:idx+9])
    
    del total_frame_label_list[0:4]
    del total_frame_label_list[-4:]
    
    for label in total_frame_label_list:
        trimmed_label.append(label)
        

In [88]:
####data info###
print(f'sample_num : {len(trimmed_data)}')
print(f'timesteps : {len(trimmed_data[0])}')
print(f'input_size : {len(trimmed_data[0][0])}')
##########

print('\n')
###label###
print(f'sample_num : {len(trimmed_label)}')
print(f'label_size : {len(trimmed_label[0])}')

########

sample_num : 45834
timesteps : 9
input_size : 26


sample_num : 45834
label_size : 4


In [93]:
'''
# data 저장된 곳
trimmed_data : list
trimmed_label : list

trimmed_data : sample_size x timesteps(9) x input_size(26)
trimmed_label : sample_size x label_size(4)
'''

'\n# data 저장된 곳\ntrimmed_data : list\ntrimmed_label : list\n\ntrimmed_data : sample_size x timesteps(9) x input_size(26)\ntrimmed_label : sample_size x label_size(4)\n'

# data_preprocessing

outputs, _status = cell(output)<br>

(모든 timesteps에 대한 결과)<br>
outputs -> [bactch x timesteps x output]<br><br>
(마지막 timestpes에 대한 결과)<br>
_status -> [1 x batch x output]<br>

# data to tensor

In [94]:
input_data_tensor = torch.tensor(trimmed_data, dtype=torch.float32)
label_tensor = torch.tensor(trimmed_label, dtype=torch.long)

In [95]:
print(input_data_tensor.shape)
print(label_tensor.shape)

torch.Size([45834, 9, 26])
torch.Size([45834, 4])


# hyper-parameter

In [100]:
input_size = 26
output_size = 4
hidden_size = 128
batch_size = 32
num_epochs = 10
learning_rate = 0.001

In [106]:
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [109]:
dataset = TensorDataset(input_data_tensor, label_tensor)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [108]:
print(dataset[0][0])
print(dataset[0][1])

tensor([[0.9855, 0.9206, 0.9876, 0.9615, 0.9881, 0.9741, 0.9948, 0.9856, 0.8109,
         0.9170, 0.7052, 0.9759, 0.7199, 0.9997, 0.6219, 0.8910, 0.9188, 0.9825,
         0.7372, 0.9853, 0.9923, 0.9718, 0.9824, 0.9717, 0.9924, 0.9788],
        [0.9856, 0.9594, 0.9877, 0.9617, 0.9882, 0.9740, 0.9949, 0.9856, 0.8109,
         0.9174, 0.7052, 0.9537, 0.7199, 0.9996, 0.6350, 0.9056, 0.9212, 0.9859,
         0.7373, 0.9852, 0.9897, 0.9719, 0.9799, 0.9717, 0.9949, 0.9788],
        [0.9857, 0.9204, 0.9877, 0.9614, 0.9881, 0.9739, 0.9949, 0.9856, 0.8228,
         0.9167, 0.7052, 0.9529, 0.7200, 0.9996, 0.6350, 0.9058, 0.9212, 0.9860,
         0.7374, 0.9853, 0.9922, 0.9718, 0.9799, 0.9717, 0.9925, 0.9789],
        [0.9857, 0.9207, 0.9878, 0.9616, 0.9881, 0.9742, 0.9949, 0.9856, 0.8228,
         0.9169, 0.7053, 0.9531, 0.7200, 0.9998, 0.6348, 0.9056, 0.9188, 0.9860,
         0.7375, 0.9999, 0.9896, 0.9718, 0.9799, 0.9717, 0.9950, 0.9788],
        [0.9856, 0.9202, 0.9877, 0.9611, 0.9882, 0.9743,

## network 정의

In [49]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.linear(hidden_size, output_size,)

    def forward(self, x):
        outputs, _ = self.lstm(x)
        #out : batch x timesteps x output_size
        out = self.fc(out[: -1, :])
        return torch.sigmoid(out)

## hyper-parameter 정의

In [ ]:
# batch x timesteps x input_size -> ? x 9 x 26
input_size  = 26
hidden_size = 128
output_size = 4
learning_rate = 0.001 
batch_size = 4

epoch_size = 1000

## criterian && optimizer 정의

In [ ]:
net = LSTM(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(net.parameters(), learning_rate)

## training network

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


In [55]:
def train_network(model, train_loader, num_epochs, criterion, optimizer):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
    epoch_loss = running_loss / len(train_loader.dataset)
    if(epoch % 100 == 0):
        print(f"Epoch [{epoch}/{num_epochs}], Loss: {epoch_loss:.4f}")

In [ ]:
def evaluate(model, test_loader, criterion, threshold):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()

    accuracy = 0.0
    precision = 0.0
    recall = 0.0
    f1 = 0.0

    test_loss = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            


    # 다중 레이블 분류 평가 지표 계산, macro : 각 클래스에 대해 개별적으로 평가
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    print(f'Average Test Loss: {avg_loss:.4f}')
    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')

## test_space

In [237]:
t = all_subject_data[0]

[0.9854827121246421,
 0.920604914933838,
 0.9875910236147845,
 0.9614620372307048,
 0.9881225954710944,
 0.9740866046403633,
 0.9948452268533359,
 0.9855752728770509,
 0.8108682923469033,
 0.9170487349295282,
 0.7052079064617098,
 0.9759499409069567,
 0.7198996276802951,
 0.9997064038501393,
 0.6218969393842055,
 0.8909792391950413,
 0.9188043028547785,
 0.9825343018563357,
 0.7371728076491344,
 0.9852939638311007,
 0.9922540327168653,
 0.9718444744676525,
 0.9823750385948061,
 0.9717275959609485,
 0.992447090747254,
 0.9788218606370539]

In [50]:
each_video = all_subject_data[0]
video_len = len(each_video)
total_frame_features_list = []
total_frame_label_list = []

t_trimmed_data = []
t_trimmed_label = []
for idx in range(0, video_len):
    total_frame_features_list.append(each_video[idx][0])
    total_frame_label_list.append(each_video[idx][1])
    
    
for idx in range(0, len(total_frame_features_list) - 8):
    t_trimmed_data.append(total_frame_features_list[idx:idx + 9])


In [38]:
t_data = []
for idx in range(0, len(total_frame_features_list) - 8):
    t_data.append(total_frame_features_list[idx:idx + 9])

In [51]:
len(t_trimmed_data)

1055

In [40]:
np.array(t_data[0])

array([[0.98548271, 0.92060491, 0.98759102, 0.96146204, 0.9881226 ,
        0.9740866 , 0.99484523, 0.98557527, 0.81086829, 0.91704873,
        0.70520791, 0.97594994, 0.71989963, 0.9997064 , 0.62189694,
        0.89097924, 0.9188043 , 0.9825343 , 0.73717281, 0.98529396,
        0.99225403, 0.97184447, 0.98237504, 0.9717276 , 0.99244709,
        0.97882186],
       [0.98563035, 0.95936408, 0.98771679, 0.96165813, 0.98816563,
        0.97402027, 0.99489721, 0.9855716 , 0.81091135, 0.91741665,
        0.70516431, 0.95371315, 0.71994492, 0.99957975, 0.63497453,
        0.90563174, 0.92120914, 0.98594924, 0.73730697, 0.98518937,
        0.98968955, 0.97188273, 0.97990497, 0.97173477, 0.99492728,
        0.97883029],
       [0.98572877, 0.92044172, 0.98767487, 0.9613836 , 0.9881226 ,
        0.97385442, 0.99487122, 0.98558631, 0.82275091, 0.91665252,
        0.70520791, 0.95289556, 0.71999022, 0.99959702, 0.63502042,
        0.90579095, 0.92121776, 0.9859887 , 0.73735169, 0.98525213,
      